# Pulse-based Variational Quantum Eigensolver Algorithm


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline
**Note: Running the program of this tutorial may cost more than 50 credit points in your Quantum Hub account.**

This tutorial introduces how to implement the variational quantum eigensolver algorithm at the pulse level. The outline of this tutorial is as follows:
- Introduction to variational quantum eigensolver
- Introduction to Pulse-Based variational quantum eigensolver
- Preparation
- Construct Hamiltonian
- Optimize two-qubit gates
- Construct problem Hamiltonian
- Pulse-based ansatz and optimization
- Summary

## Introduction to variational quantum eigensolver

Variational Quantum Eigensolver (VQE) is a popular quantum algorithm for approximating the ground state energy of molecules on Noisy Intermediate-Scale Quantum (NISQ) computers. VQE is mainly used to estimate the minimum eigenvalue of a given Hamiltonian and find its ground state. For near-term quantum computers, critical gate errors, decoherence, and poor connectivity limit the depth of quantum circuits. However, the VQE algorithm can work with low-depth quantum circuits. Hence, the VQE algorithm is considered an ideal candidate to utilize NISQ devices to solve real-world problems.

In VQE, the essential task is to calculate the ground state energy of a given discretized molecular Hamiltonian $\hat{H}_{\rm mole}$ by a parametrized trail state $|\psi(\vec{\theta})\rangle$. The trail state $|\psi(\vec{\theta})\rangle$ is generated by a parameterized ansatz. At the same time, we employ the classical optimization methods to find a set of optimal $\vec{\theta}$ to minimize the expectation value $E = \langle \psi(\vec{\theta}) | \hat{H}_{\rm mole} | \psi(\vec{\theta}) \rangle$, which represents the approximate ground state energy $E_0$ of the Hamiltonian $\hat{H}_{\rm mole}$:
$$
E_0 = {\rm min}_{\vec{\theta}} \langle \psi(\vec{\theta}) | \hat{H}_{\rm mole} | \psi(\vec{\theta}) \rangle
$$

In this tutorial, we will introduce the method to implement VQE for solving the ground state energy of a hydrogen molecule $H_2$. We simulate the VQE on a superconducting system at the pulse level while considering multiple nonideal factors. The following figure shows a common ansatz for generating trail states:

![VQE](figures/vqe-circuit.png)

In this example, we no longer use CNOT gates, because it is not directly implemented in the superconducting platform. However, we use the superconducting hardware-efficient two-qubit gates, i.e., cross-resonance (CR) gates, which also can realize maximal entanglements using single-qubit gates. Thus, the above four-qubit circuit is formed by single-qubit gates and CR gates. The matrix of an ideal CR gate is:

$$
\hat{U}_{\rm CR}(\alpha) = \begin{bmatrix}
\cos{\frac{\alpha}{2}} & -i\sin{\frac{\alpha}{2}} & 0 & 0 \\
-i\sin{\frac{\alpha}{2}} & \cos{\frac{\alpha}{2}} & 0 & 0 \\ 
0 & 0 & \cos{\frac{\alpha}{2}} & i\sin{\frac{\alpha}{2}} \\
0 & 0 & i\sin{\frac{\alpha}{2}} & \cos{\frac{\alpha}{2}} 
\end{bmatrix}.
$$

Here, we choose $\alpha = -\pi/2$. More details about the CR gate can be found [here](https://quanlse.baidu.com/#/doc/tutorial-cr). 

## Introduction to Pulse-Based variational quantum eigensolver

In this tutorial, we study the VQE algorithm at the pulse level, which we call **pulse-based VQE**. Unlike the classical VQE which optimizes the rotation parameters of the single-qubit gates from the logical quantum circuit, pulse-based VQE directly takes the pulse parameters as the optimization parameters to find the minimal loss value (i.e. ground state energy). The following figure shows the difference between the pulse-based VQE and the standard VQE:

![VQE](figures/vqe-scheme.png)

To implement pulse-based VQE, we need to translate the logical quantum circuit into a **pulse-based quantum circuit**, i.e., the logical rotating gates $R_x(\theta)$ and $R_y(\theta)$ are replaced by the corresponding control pulses on $X$ and $Y$ channels respectively with variable amplitudes - which we call **pulse-based gates**:

![VQE](figures/vqe-translate.png)

In this figure, $U_{\rm ENT}$ is a unitary matrix which can create the required entanglement - the details will be explained in the following sections. Here, we use a new notation representing the parameters of **pulse-based gates**:
$$
\vec{A} = [A_0, \cdots, A_m, \cdots, A_{M-1}],
$$
where $M$ is the total number of **pulse-based gates**; $A_m$ denotes the amplitude of the Gaussian waveform of the $m$-th **pulse-based gate**:
$$
\Omega_m(t) = A_m e^{-(\frac{t - \tau_m}{\sqrt{2} \sigma_m}) ^2}.
$$
Other pulse parameters such as the width $\sigma_m$ and the central position $\tau_m$ will be fixed throughout the procedure. Based on the introduction of **pulse-based VQE**, we no longer need to generate an optimized drive pulse according to the parameters from the logical quantum circuit. This improves the efficiency of VQE and the accuracy of the results significantly.

In the above sections, we give a brief introduction to the standard and pulse-based VQE. In the following sections, we will demonstrate the implementation of pulse-based VQE using Quanlse step by step.

## Preparation
After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# This module creates the Hamiltonian dictionary
from Quanlse.QHamiltonian import QHamiltonian

# These functions help us perform matrix calculations
from Quanlse.Utils.Functions import tensor
from Quanlse.Utils.Infidelity import unitaryInfidelity

# These functions define useful operator matrices
from Quanlse.QOperator import sigmaX, sigmaY, sigmaZ, sigmaI

# This function generates wave data
from Quanlse.QWaveform import QJob, QJobList, QWaveform, gaussian, square

# This function uploads jobs to Quanlse Cloud Service and receives results
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# This module defines matrices of the frequently used quantum gates
from Quanlse.QOperation import FixedGate

# This module saves the PBVQE results
from Quanlse.Define import outputPath

In [ ]:
# Import the necessary packages
import os
from numpy import linalg, min, random, savez, load, identity, kron, zeros
from math import pi
from functools import reduce
from scipy import optimize

# Generate the path of npz file
localFile = os.path.join(outputPath, f'pbvqe.npz')

## Construct Hamiltonian

First, we define some necessary constants, including the sampling period of the arbitrary wave generator (AWG), the number of qubits in the circuit, and the system's energy levels to consider.

In [ ]:
# Sampling period (Nano second)
dt = 2.0

# Number of qubits
qubits = 4

# System energy level
level = 2

Then, we define the hardware parameters of the superconducting qubits. The items in the `freq` list are $\omega_{\rm q0}, \omega_{\rm q1}, \omega_{\rm q2}, \omega_{\rm q3}$; the items in the `coupling` list save the coupling information of qubit 0-1, 1-2, 2-3, 3-0, respectively. By using the RWA(Rotating Wave Approximation), we define the system in a rotaing frame with the qubits' frequencies at: $\omega_{\rm RWA} = \omega_{\rm q0} = \omega_{\rm q2} = 4.914 \times 2\pi$ GHz.

In [ ]:
# Define the hardware parameters of the qubits (GHz)
freq = [4.914 * (2 * pi), 5.114 * (2 * pi), 4.914 * (2 * pi), 5.114 * (2 * pi)]

# Define the coupling strength (GHz)
coupling = [
    [[0, 1], 0.016 * (2 * pi)],
    [[1, 2], 0.016 * (2 * pi)],
    [[2, 3], 0.016 * (2 * pi)],
    [[3, 0], 0.016 * (2 * pi)]
]

# Frequency of rotating frame (GHz)
rwa = 4.914 * (2 * pi)

Here, we fix the gate time for all of the single-qubit and two-qubit gates:

In [ ]:
# Gate time (Nano second)
tg2q = 200
tg1q = 64

Then, we create the Hamiltonian according to the following hardware structure: each qubit couples with its neighbors and the coupling strength is always constant.

![VQE](figures/vqe-topo_structure.png)

Here, we construct the Hamiltonian of the system above:
$$
\hat{H}_{\rm total} = \sum_{q=0}^{3} \delta_{q} \hat{a}^{\dagger}_{q}\hat{a}_{q} + \frac{1}{2}\sum_{q=0}^{3}g_{q,q+1}(\hat{a}_{q}\hat{a}^{\dagger}_{q+1}+\hat{a}^{\dagger}_{q}\hat{a}_{q+1}) + \sum_{q=0}^{3}\Omega_{q}^x (t) \hat{\sigma}_{q}^{x} + \sum_{q=0}^{3}\Omega_{q}^y (t) \hat{\sigma}_{q}^{y} + \sum_{q=0}^{3}\Omega_{q}^z (t) \hat{\sigma}_{q}^{z} ,
$$
where $\hat{a}_{q}$ and $\hat{a}^{\dagger}_{q}$ are the annihilation and creation operators on the $q$-th qubit respectively. $\hat{\sigma}^x_{q}, \hat{\sigma}^y_{q}$ and $\hat{\sigma}^z_{q}$ are the Pauli operators on the $q$-th qubit. $\delta_{q}=\omega_{q} - \omega_{\rm RWA}$ is the detuning of the $q$-th qubit；$g_{q,q+1}$ are the coupling strength between $q$-th and $q+1$-th qubit $\Omega_q^{x,y,z}(t)$ are the envelope functions of the drive pulses or of the magnetic flux control. In Quanlse, the Hamiltonian above can be constructed as follows:

In [ ]:
# Create the Hamiltonian
vqeHam = QHamiltonian(qubits, level, dt)

# Add the coupling terms
for item in coupling:
    q0, q1 = item[0][0], item[0][1]
    vqeHam.addCoupling([q0, q1], g=item[1] / 2)

for qubit in range(qubits):
    # Add the detuning terms
    detuning = freq[qubit] - rwa
    vqeHam.addDrift(sigmaZ(), qubit, coef=detuning)

For more information on constructing Hamiltonian using Quanlse, please see [tutorial-single-qubit-gate](https://quanlse.baidu.com/#/doc/tutorial-single-qubit).

## Optimize two-qubit gates

In this example, we use cross-resonance gates as the entangling gates. For more information on the CR gate, please see [tutorial-cr-gate](https://quanlse.baidu.com/#/doc/tutorial-cr). Due to the direct coupling of the neighboring qubits, applying X pulses on one qubit will end up affecting two qubits. Hence, we need to take this factor into account when we design our pulses in order to suppress the unwanted effects. 

![VQE](figures/vqe-crosstalk.png)

Here, we use `vqeHam.subSystem()` to extract two three-qubit sub-systems from `vqeHam` for optimizing the CR gates - one of which is qubits 0-1-2 and the other is qubits 1-2-3. On these sub-systems, we set $\hat{U}_{\rm goal}=I \otimes \hat{U}_{\rm CR}$ as the goal unitary matrix to optimize the pulses' parameters - the goal is to generate a CR gate on the second and the third qubits of the sub-system.

We define a function `makeCrPulse()` to define the waveforms. The Gaussian microwave drive pulses are applied to the second qubits of the sub-systems. We fix the width and the center position of the Gaussian wave while setting the amplitude as the first parameter to optimize. The second parameter to be optimized is the amplitude of the flux detuning (square wave) applied on the first qubit. Note that the the drive terms added with `tag="det"` are used to transform the rotating frame to a particular frequency.

In [ ]:
def makeCrPulse(ham, subSys3q, driveFreq, amp, shift, t):
    """ Assemble the pulses for CR gates """
    subHam = ham if subSys3q is None else ham.subSystem(subSys3q)
    subHam.clearWaves()
    subHam.addWave(sigmaX(), 1, gaussian(0, t, amp, tg2q / 2, tg2q / 8), tag="XY")
    # frame transformation
    subHam.addWave(sigmaZ(), 0, square(0, t, rwa - driveFreq + shift), tag="Z")
    subHam.addWave(sigmaZ(), 1, square(0, t, rwa - driveFreq), tag="det")
    subHam.addWave(sigmaZ(), 2, square(0, t, rwa - driveFreq), tag="det")
    return subHam.job if subSys3q is None else subHam.outputInverseJob(qubits)

Then, we define a function `optimizeCr` to perform the optimization. The optimal parameters will be saved for further usage.

In [ ]:
def optimizeCr(subSys3q, driveFreq):
    """ Realize a CR gate on the second & third qubits """
    crHam = vqeHam.subSystem(subSys3q)
    uGoal = tensor([identity(2), FixedGate.CR.getMatrix()])

    def crLoss(_x):
        # Clear and add waves
        crHam.clearWaves()
        # Generate and add waves for CR gate implementation
        _crJob = makeCrPulse(crHam, None, driveFreq, _x[0], _x[1], tg2q)
        # Simulate the system's evolution and obtain the infidelity
        unitary = crHam.simulate(job=_crJob)[0]["unitary"]
        infidelity = unitaryInfidelity(uGoal, unitary, 3)
        return infidelity

    opt = optimize.dual_annealing(crLoss, [(-2, 2), (-0.2, 0.2)], maxiter=60)
    print("Min infidelity:", opt["fun"])
    return opt["x"][0], opt["x"][1]

lhlQ1X, lhlQ0Z = optimizeCr([0, 1, 2], 4.914 * 2 * pi)
hlhQ1X, hlhQ0Z = optimizeCr([1, 2, 3], 5.114 * 2 * pi)

## Construct problem Hamiltonian

In this section, we introduce the method for estimating the ground state energy for hydrogen molecule $H_2$ at the pulse level. Here we will skip the details on fermion-to-qubit mapping, please visit [Paddle Quantum](https://github.com/PaddlePaddle/Quantum/blob/master/tutorial/quantum_simulation/VQE_EN.ipynb) for more information. First, we define a function `pauli_str_to_matrix()` to convert the Pauli string to the Hamiltonian $\hat{H}_{\rm mole}$:

In [ ]:
def pauliStrToMatrix(pauli_str, n):
    """
    Convert the Pauli string in Hamiltonian
    """
    def NKron(AMatrix, BMatrix, *args):
        return reduce(
            lambda result, index: kron(result, index),
            args,
            kron(AMatrix, BMatrix), )
    pauli_dict = {
        'i': sigmaI().matrix,
        'x': sigmaX().matrix,
        'y': sigmaY().matrix,
        'z': sigmaZ().matrix
    }
    # Parse pauli_str; 'x0,z1,y4' to 'xziiy'
    new_pauli_str = []
    for coeff, op_str in pauli_str:
        init = list('i' * n)
        op_list = op_str.split(',')
        for op in op_list:
            pos = int(op[1:])
            assert pos < n, 'n is too small'
            init[pos] = op[0]
        new_pauli_str.append([coeff, ''.join(init)])

    # Convert new_pauli_str to matrix; 'xziiy' to NKron(x, z, i, i, y)
    matrices = []
    for coeff, op_str in new_pauli_str:
        sub_matrices = []
        for op in op_str:
            sub_matrices.append(pauli_dict[op])
        if len(op_str) == 1:
            matrices.append(coeff * sub_matrices[0])
        else:
            matrices.append(coeff * NKron(sub_matrices[0], sub_matrices[1], *sub_matrices[2:]))

    return sum(matrices)

Then we use the Hamiltonian of a $H_2$ molecule with an interatomic distance of $d=74$ pm. The following data is generated from [Paddle Quantum](https://github.com/PaddlePaddle/Quantum/blob/master/tutorial/quantum_simulation/VQE_EN.ipynb).

In [ ]:
targetHam = [
    [-0.042078976477822, 'i0'],
    [ 0.177712874651399, 'z0'],
    [ 0.177712874651399, 'z1'],
    [-0.242742805131446, 'z2'],
    [-0.242742805131462, 'z3'],
    [ 0.170597383288005, 'z0,z1'],
    [ 0.044750144015351, 'y0,x1,x2,y3'],
    [-0.044750144015351, 'y0,y1,x2,x3'],
    [-0.044750144015351, 'x0,x1,y2,y3'],
    [ 0.044750144015351, 'x0,y1,y2,x3'],
    [ 0.122933050561837, 'z0,z2'],
    [ 0.167683194577189, 'z0,z3'],
    [ 0.167683194577189, 'z1,z2'],
    [ 0.122933050561837, 'z1,z3'],
    [ 0.176276408043195, 'z2,z3']
]
hMatrix = pauliStrToMatrix(targetHam, 4)

The theoretical ground state energy is:

In [ ]:
# Calculate the theoretical eigenvalue
eigVal, eigState = linalg.eig(hMatrix)
minEigH = min(eigVal.real)
print(f"Ground state energy: {minEigH} Ha")

## Pulse-based ansatz and optimization

We design the pulse-based VQE ansatz using one of the most commonly-used templates for a $H_2$ molecule. The following figure shows one layer of the ansatz, containing 3 single-qubit gates for each qubit. Each single-qubit gate holds one parameter as the amplitude of a Gaussian envelope, in which the width and the center position are fixed.

![VQE](figures/vqe-scheduling.png)

Here, we define a function `makeWaveSchedule()` to schedule the pulse sequence according to the pulse-based ansatz shown above. Argument `x` is a list of optimization parameters (i.e., pulse parameters $\vec{A}$); `vqeJob` is a list of waveform data generated by `addWave()`, which saves the detailed information of the user-defined waveform.

In [ ]:
def makeWaveSchedule(x):
    """ Generate waves for pulse-based circuit """
    # Generate pulses for CR gate
    crJob = vqeHam.createJob()
    crJob += makeCrPulse(vqeHam, [3, 0, 1], 5.114 * 2 * pi, hlhQ1X, hlhQ0Z, tg2q)
    crJob += makeCrPulse(vqeHam, [0, 1, 2], 4.914 * 2 * pi, lhlQ1X, lhlQ0Z, tg2q)
    crJob += makeCrPulse(vqeHam, [1, 2, 3], 5.114 * 2 * pi, hlhQ1X, hlhQ0Z, tg2q)
    crJob += makeCrPulse(vqeHam, [2, 3, 0], 4.914 * 2 * pi, lhlQ1X, lhlQ0Z, tg2q)
    # Assemble the pulses
    depth = int(len(x) / 12)
    vqeJob = vqeHam.createJob()
    for d in range(depth):
        gate1QJob = vqeHam.createJob()
        # Add pulses for single-qubit gates
        for q in range(4):
            # X/Y/X controls
            gate1QJob.addWave(sigmaX(), q, gaussian(0, tg1q, x[12 * d + q], tg1q / 2, tg1q / 8))
            gate1QJob.addWave(sigmaY(), q, gaussian(tg1q, tg1q, x[12 * d + 4 + q], tg1q / 2, tg1q / 8))
            gate1QJob.addWave(sigmaX(), q, gaussian(tg1q * 2, tg1q, x[12 * d + 8 + q], tg1q / 2, tg1q / 8))
            # Set detuning
            gate1QJob.addWave(sigmaZ(), q, square(0, tg1q * 3, rwa - freq[q]), tag="det")
        vqeJob += gate1QJob
        vqeJob += crJob
    return vqeJob

In this example, we use the gradient-based optimization method (L-BFGS-B) provided by `Scipy`. In this method, information regarding the gradient for all parameters must be provided. We use two-point finite difference method to approximate the gradient:
$$
\frac{\partial{\rm Loss}(\vec{A})}{\partial A_m} = \frac{{\rm Loss}(A_0, \cdots, A_m + \epsilon, \cdots, A_{M-1}) - {\rm Loss}(A_0, \cdots, A_m - \epsilon, \cdots, A_{M-1})}{2\epsilon} ,
$$
where $\vec{A} = [A_0, \cdots, A_{M-1}]$ is the pulse parameter list, $\epsilon$ is a small number, and the ${\rm Loss}(\vec{A})$ is defined as,
$$
{\rm Loss}(\vec{A}) =  \langle \psi(\vec{A}) | \hat{H}_{\rm mole} | \psi(\vec{A}) \rangle.
$$
Trail state $\psi(\vec{A})$ is generated by the pulse-based ansatz. Finite difference requires a large number of samples. For example, given the number of pulse parameter $M$, we need $2M$ samples to estimate the approximated gradient. Hence, we use Quanlse Cloud Service to accelerate this procedure. 

To utilize Quanlse Cloud Service, we need to import `Define` and acquire a token, which can be obtained on [Baidu Quantum-Hub](http://quantum-hub.baidu.com).

In [ ]:
# Define the loss function
import copy
from Quanlse import Define
Define.hubToken = ""

Then, we define the VQE `loss` function. In this function, we simulate the evolution of the pulse-based circuit at $\vec{x}$ (the amplitude parameters of pulse-based gates) and compute the gradient at this point by the finite difference method we mentioned above. In each iteration, we input the current pulse parameter list $\vec{x}$ into the loss function and package all samples into `waveList`. `waveList` contains $2M$ samples for solving the gradient and $1$ sample for obtaining the loss value. 

Finally, we integrate all the waves into one list, i.e., `waveList`, and submit it to Quanlse Cloud Service by function `runHamiltonian()`. After about 15 to 20 seconds, we receive the result, which will be saved to the `Output` folder as a JSON file. At the same time, the variable `result` will be assigned a list that contains all the simulation results corresponding to the `waveList`.

**Note:** Each item of `waveList` contains all the waves for pulse-based VQE, generated by the function `makeWaveSchedule()` we have defined above.

In [ ]:
def loss(x):
    global lossHistory
    # Add wave for current point
    waveList = vqeHam.createJobList()
    waveList.addJob(makeWaveSchedule(x))
    
    # Add wave for calculating gradient
    for xId in range(len(x)):
        xList = copy.deepcopy(x)
        xList[xId] -= 1e-8
        waveList.addJob(makeWaveSchedule(xList))
        xList[xId] += 2 * 1e-8
        waveList.addJob(makeWaveSchedule(xList))

    # Simulate the evolution
    result = runHamiltonian(vqeHam, jobList=waveList)

    # Calculate the loss function
    lossList = []
    for item in result:
        state = item["unitary"]
        lossVal = (state.conj().T @ hMatrix @ state).real[0][0]
        lossList.append(lossVal)
    
    # Calculate the gradients
    gradient = []
    for index in range(len(x)):
        gradient.append((lossList[2 + 2 * index] - lossList[1 + 2 * index]) / 1e-8 / 2)
    
    print("Loss function:", lossList[0])
    lossHistory.append(lossList[0])
    return lossList[0], gradient

Then we use `fmin_l_bfgs_b()` provided by `Scipy` to minimize the loss function, which corresponds to the ground state energy. We start with randomly initialized parameters. In this example, we set the ansatz depth $d=3$, define the bounds for all pulse parameters $\vec{x}$ and limit the maximal numbers of iteration.

**Note**: this optimization may take longer than 15 minutes.

In [ ]:
depth = 3
lossHistory = []
initParas = [random.rand() for _ in range(depth * 12)]
bounds = [(-1.5, 1.5) for _ in range(depth * 12)]
x, f, d = optimize.fmin_l_bfgs_b(loss, initParas, fprime=None, bounds=bounds, maxiter=200)

# Save the loss history to a file for further usage
savez(localFile, lossHistory)

In [ ]:
print(f"The estimated ground state energy is: {f} Ha")
print("Total iteration:", d["nit"])

As we see, the optimization converges at high precision, and the final number of iteration is 58.

Then, we plot the optimized energy values to the iteration number:

In [ ]:
# Load the loss_history list from the npz file.
lossHistory = load(localFile)['arr_0']

# Plot the figures
import matplotlib.pyplot as plt
plt.plot(range(len(lossHistory)), lossHistory, label="Energy")
plt.axhline(minEigH, c="gray", ls="--", lw=1.0)
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.show()

Finally, we can plot the pulse sequence using the `plotWaves()` function:

In [ ]:
# Print the waveforms.
makeWaveSchedule(x).plot(color=['red', 'green', 'blue'])

## Summary
This tutorial introduces how to implement the variational quantum eigensolver algorithm at the pulse level. Users are encouraged to explore other advanced research which are different from this tutorial.

## References

\[1\] [Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." *Nature communications* 5 (2014): 4213.](https://doi.org/10.1038/ncomms5213)

\[2\] [Moll, Nikolaj, et al. "Quantum optimization using variational algorithms on near-term quantum devices." *Quantum Science and Technology* 3.3 (2018): 030503.](https://doi.org/10.1088/2058-9565/aab822)

\[3\] [Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." *Nature* 549.7671 (2017): 242-246.](https://doi.org/10.1038/nature23879)

\[4\] [Rigetti, Chad, and Michel Devoret. "Fully microwave-tunable universal gates in superconducting qubits with linear couplings and fixed transition frequencies." *Physical Review B* 81.13 (2010): 134507.](https://doi.org/10.1103/PhysRevB.81.134507)

\[5\] [Meitei, Oinam Romesh, et al. "Gate-free state preparation for fast variational quantum eigensolver simulations: ctrl-VQE." *arXiv preprint arXiv:2008.04302 (2020)*.](https://arxiv.org/abs/2008.04302)

\[6\] [Wilhelm, Frank K., et al. "An introduction into optimal control for quantum technologies." *arXiv preprint arXiv:2003.10132* (2020).](https://arxiv.org/abs/2003.10132)

\[7\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)